In [310]:
%use kandy
%use lib-ext

In [311]:
@file:DependsOn("com.ustermetrics:ecos4j:2.0.0")
@file:DependsOn("com.ustermetrics:ecos4j-native:1.0.4-2.0.10")
@file:DependsOn("org.ejml:ejml-all:0.43.1")

In [312]:
import com.ustermetrics.ecos4j.Model
import com.ustermetrics.ecos4j.Parameters
import com.ustermetrics.ecos4j.Status.OPTIMAL
import org.ejml.data.DMatrixSparseCSC
import org.ejml.dense.row.factory.DecompositionFactory_DDRM
import org.ejml.ops.DConvertMatrixStruct
import org.ejml.simple.SimpleMatrix

In [313]:
// Define portfolio optimization problem
val mu = SimpleMatrix(doubleArrayOf(0.05, 0.06, 0.08, 0.06))
val sigma = SimpleMatrix(
    4, 4, true,
    0.0225, 0.003, 0.015, 0.0225,
    0.003, 0.04, 0.035, 0.024,
    0.015, 0.035, 0.0625, 0.06,
    0.0225, 0.024, 0.06, 0.09
)
val sigmaLimit = 0.2

In [314]:
// Problem dimension
val n = mu.getNumRows()

In [315]:
// Compute Cholesky decomposition of sigma
val chol = DecompositionFactory_DDRM.chol(n, true)
if (!chol.decompose(sigma.getMatrix()))
    throw IllegalStateException("Cholesky decomposition failed")
val upTriMat = SimpleMatrix.wrap(chol.getT(null)).transpose()
// Define second-order cone program
val cMat = mu.negative()
    .concatRows(SimpleMatrix(1, 1))
System.out.println("\ncMat")
cMat.print()

val aMat = SimpleMatrix.ones(1, n)
    .concatColumns(SimpleMatrix(1, 1))
System.out.println("\naMat")
aMat.print()

val bMat = SimpleMatrix.ones(1, 1)
System.out.println("\nbMat")
bMat.print()

val gMatPosOrt = SimpleMatrix.identity(n)
    .negative()
    .concatColumns(SimpleMatrix(n, 1))
    .concatRows(SimpleMatrix(1, n).concatColumns(SimpleMatrix.ones(1, 1)))
val gMatSoc = SimpleMatrix(1, n)
    .concatColumns(SimpleMatrix.filled(1, 1, -1.0))
    .concatRows(upTriMat.negative().concatColumns(SimpleMatrix(n, 1)))
val gMat = gMatPosOrt.concatRows(gMatSoc)
System.out.println("\ngMat")
gMat.print()

val hMat = SimpleMatrix(2 * n + 2, 1)
hMat.set(n, 0, sigmaLimit)
System.out.println("\nhMat")
hMat.print()


cMat
Type = DDRM , rows = 5 , cols = 1
-.05       
-.06       
-.08       
-.06       
 0         

aMat
Type = DDRM , rows = 1 , cols = 5
 1           1           1           1           0         

bMat
Type = DDRM , rows = 1 , cols = 1
 1         

gMat
Type = DDRM , rows = 10 , cols = 5
-1          -0          -0          -0           0         
-0          -1          -0          -0           0         
-0          -0          -1          -0           0         
-0          -0          -0          -1           0         
 0           0           0           0           1         
 0           0           0           0          -1         
-.15        -.02        -.1         -.15         0         
-0          -.198997487 -.16583124  -.105528971  0         
-0          -0          -.158113883 -.173925271  0         
-0          -0          -0          -.161597142  0         

hMat
Type = DDRM , rows = 10 , cols = 1
 0         
 0         
 0         
 0         
 .2        
 0    

In [316]:
// ecos4j needs sparse aMat and gMat
val tol = 1e-8
val aSpMat = DConvertMatrixStruct.convert(aMat.getDDRM(), null as DMatrixSparseCSC?, tol)
System.out.println("\naSpMat")
aSpMat.print()

val gSpMat = DConvertMatrixStruct.convert(gMat.getDDRM(), null as DMatrixSparseCSC?, tol)
System.out.println("\ngSpMat")
gSpMat.print()


aSpMat
Type = DSCC , rows = 1 , cols = 5 , nz_length = 4
 1           1           1           1               *     

gSpMat
Type = DSCC , rows = 10 , cols = 5 , nz_length = 16
-1               *           *           *           *     
     *      -1               *           *           *     
     *           *      -1               *           *     
     *           *           *      -1               *     
     *           *           *           *       1         
     *           *           *           *      -1         
-.15        -.02        -.1         -.15             *     
     *      -.198997487 -.16583124  -.105528971      *     
     *           *      -.158113883 -.173925271      *     
     *           *           *      -.161597142      *     


In [317]:
// Helper function
fun toLongArray(arr: IntArray): LongArray {
    return arr.map { it.toLong() }.toLongArray()
}

In [318]:
var xMat: SimpleMatrix

Model().use {
    // Set up model
    it.setup(
        n + 1L, longArrayOf(n + 1L), 0, gSpMat.nz_values, toLongArray(gSpMat.col_idx),
        toLongArray(gSpMat.nz_rows), cMat.getDDRM().data, hMat.getDDRM().data, aSpMat.nz_values,
        toLongArray(aSpMat.col_idx), toLongArray(aSpMat.nz_rows), bMat.getDDRM().data
    )

    // Create and set parameters
    val parameters = Parameters.builder()
        .verbose(true)
        .build()
    it.setParameters(parameters)

    // Optimize model
    val status = it.optimize()
    if (status != OPTIMAL)
        throw IllegalStateException("Optimization failed")

    // Get solution
    xMat = SimpleMatrix(it.x())
    System.out.println("xMat")
    xMat.print()
}

xMat
Type = DDRM , rows = 5 , cols = 1
 .248790206
 .049684806
 .701524985
 3.5308E-09
 .2        


$$
\begin{align*}
& \text{maximize} & & \mu^T x \\
& \text{subject to} & & x^T \Sigma x \leq \sigma^2 \\
& & & \mathbf{1} x = 1 \\
& & & x \geq 0
\end{align*}
$$

In [319]:
// Plot solution
val assets = MutableList(n) { i -> "Asset ${i + 1}" }
val percentages = MutableList(n) { i -> xMat.get(i, 0) }

plot {
    bars {
        x(assets) { axis.name = "" }
        y(percentages) { axis.name = "Percentage Allocation (%)" }
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="tRKc1G"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"name":""
},{
"aesthetic":"y",
"name":"Percentage Allocation (%)",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":["Asset 1","Asset 2","Asset 3","Asset 4"],
"y":[0.24879020572072666,0.04968480618198966,0.7015249845664572,3.530816281223793E-9]
},
"sampling":"none",
"position":"dodge",
"geom":"bar"
}]
};
 var plotContainer = document.getElementById("tRKc1G");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Asset 1 
 
 
 
 
 
 
 
 
 Asset 2 
 
 
 
 
 
 
 
 
 Asset 3 
 
 
 
 
 
 
 
 
 Asset 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 
 
 
 
 0.1 
 
 
 
 
 
 
 0.2 
 
 
 
 
 
 
 0.3 
 
 
 
 
 
 
 0.4 
 
 
 
 
 
 
 0.5 
 
 
 
 
 
 
 0.6 
 
 
 
 
 
 
 0.7 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Percentage Allocation (%)